In [1]:
import pandas as pd
import numpy as np
import itertools
pd.options.mode.chained_assignment = None

#read in csv file
df = pd.read_csv('projectData.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9762 entries, 0 to 9761
Data columns (total 3 columns):
coID         9762 non-null int64
sales        9762 non-null float64
inventory    9762 non-null float64
dtypes: float64(2), int64(1)
memory usage: 228.9 KB


In [2]:
#sort values, reset index
df = df.sort_values(by=['sales'])
df = df.reset_index(drop=True)

df2 = df.copy()

# Determining Strata #1: Cumlative Method

#### Determining 10 Stratas

In [3]:
#determining percentile/strata cutpoints
df2['cumSales'] = pd.Series(df2.sales, index=df2.index)
for i in range(len(df2.sales)):
    if i > 0:
        df2.cumSales[i] = df2.sales[i] + df2.cumSales[i-1]
    else:
        df2.cumSales[i] = df2.sales[i]
print df2.head(5),"\n"
print df2.tail(5),"\n"
print "Total Sales: ",df2.sales.sum()
print "5% of Sales Cutpoint: ",df2.sales.sum()/20

   coID       sales      inventory     cumSales
0  6578  124.430200    6756.016916   124.430200
1  6531  164.460412    2429.508343   288.890613
2  4555  191.616081   46088.726420   480.506694
3  8497  210.496795    2990.580635   691.003490
4  2096  842.906821  194922.480200  1533.910311 

      coID         sales     inventory      cumSales
9757    29  8.785041e+06  6.674551e+06  1.289021e+09
9758    35  1.198820e+07  4.331299e+06  1.301009e+09
9759    18  1.651809e+07  1.016784e+08  1.317527e+09
9760    28  1.912679e+07  2.062727e+06  1.336654e+09
9761    19  5.215643e+07  1.053796e+08  1.388810e+09 

Total Sales:  1388810307.69
5% of Sales Cutpoint:  69440515.3843


In [4]:
#assigning rows to each percentile
pctl_cutpoints = np.arange(69440515.3843, 1388810307.69, 69440515.3843)
pctl = ['P00-P05','P05-P10','P10-P15','P15-P20','P20-P25','P25-P30','P30-P35','P35-P40',
       'P40-P45','P45-P50','P50-P55','P55-P60','P60-P65','P65-P70','P70-P75','P75-P80',
       'P80-P85','P85-P90','P90-P95','P95-P100']
pctl_dict = dict(itertools.izip(pctl_cutpoints, pctl))

df2['pctl'] = pd.Series(pctl[-1], index=df2.index)
for i in range(len(df2.cumSales)):
    for j in range(len(pctl_cutpoints)-1,-1,-1):
        if df2.cumSales[i] < pctl_cutpoints[j]:
            df2.pctl[i] = pctl_dict.get(pctl_cutpoints[j])     

In [11]:
strata_1_alloc = [1, 1, 2, 2, 3, 3, 4, 4, 5, 5, 6, 6, 7, 7, 8, 8, 9, 9, 10, 10]
strata_1_dict = dict(itertools.izip(pctl, strata_1_alloc))
strata_1 = []

for i in range(len(df2.pctl)):
    strata_1.append(strata_1_dict.get(df2.pctl[i]))
print len(strata_1)

9762


In [13]:
df['strata_1'] = pd.Series(strata_1, index=df.index)

# Determining Strata #2: Cumlative Sqrt Frequency Method

#### 10 Stratas

In [16]:
#determining strata cutpoints and assigning them to percentile cutpoints
pctl_freq = df2.groupby(['pctl'])['sales'].count()
pctl_freq_sqrt = np.sqrt(pctl_freq)
cum_freq_sqrt = []

for i in range(len(pctl_freq)):
    if i > 0:
        cum_freq_sqrt.append(pctl_freq_sqrt[i] + cum_freq_sqrt[i-1])
    else:
        cum_freq_sqrt.append(pctl_freq_sqrt[i])

strata_cut = pctl_freq_sqrt.sum()/10

strata = []
for i in cum_freq_sqrt:
    strata.append(round(i / strata_cut))
        
d = {'freq':pctl_freq,'sqrt_freq':pctl_freq_sqrt,
     'cum_sqrt_freq':cum_freq_sqrt,'strata':strata}
pctl_stats = pd.DataFrame(data=d)
print pctl_stats
               
print "\nTotal Cumulative sqrt(pctl_freq): ",pctl_freq_sqrt.sum()
print "Strata Cutpoint (h=4): ",pctl_freq_sqrt.sum()/10

          cum_sqrt_freq  freq  sqrt_freq  strata
pctl                                            
P00-P05       50.596443  2560  50.596443     1.0
P05-P10       92.129562  1725  41.533119     3.0
P10-P15      125.863817  1138  33.734256     4.0
P15-P20      155.018577   850  29.154759     4.0
P20-P25      181.608049   707  26.589472     5.0
P25-P30      206.123350   601  24.515301     6.0
P30-P35      228.439263   498  22.315914     6.0
P35-P40      248.957548   421  20.518285     7.0
P40-P45      266.818119   319  17.860571     7.0
P45-P50      282.147829   235  15.329710     8.0
P50-P55      295.376585   175  13.228757     8.0
P55-P60      307.166412   139  11.789826     9.0
P60-P65      317.606718   109  10.440307     9.0
P65-P70      327.198381    92   9.591663     9.0
P70-P75      336.086575    79   8.888194     9.0
P75-P80      342.868905    46   6.782330    10.0
P80-P85      348.699857    34   5.830952    10.0
P85-P90      353.495689    23   4.795832    10.0
P90-P95      356.495

In [18]:
#assigning rows to each strata
strata_dict = dict(zip(pctl,pctl_stats.strata))

strata_2 = []
for i in range(len(df2.cumSales)):
    strata_2.append(int(strata_dict.get(df2.pctl[i])))
    
print len(strata_2)

9762


In [19]:
df['strata_2'] = pd.Series(strata_2, index=df.index)

In [20]:
neymanAlloc(df, 'strata_2', 500)

   stratum   N_h       N_h*S_h  N_h*S_h/sum(N_h*S_h)  \
0        1  2560  2.328536e+07              0.029733   
1        3  1725  4.475844e+06              0.005715   
2        4  1988  2.395574e+07              0.030589   
3        5   707  3.076412e+06              0.003928   
4        6  1099  1.500434e+07              0.019159   
5        7   740  2.311884e+07              0.029521   
6        8   410  2.346314e+07              0.029960   
7        9   419  6.390716e+07              0.081604   
8       10   114  6.028519e+08              0.769789   

   samp*N_h*S_h/sum(N_h*S_h)  sample size  
0                  14.866686           15  
1                   2.857631            3  
2                  15.294700           15  
3                   1.964155            2  
4                   9.579621           10  
5                  14.760373           15  
6                  14.980193           15  
7                  40.801942           41  
8                 384.894700          385  

# Determining Strata #3: Equal W_h S_h Method

#### 10 Stratas

In [22]:
df3 = df[['coID','sales','inventory','strata_2']].copy()
print df3.head(5)

   coID       sales      inventory  strata_2
0  6578  124.430200    6756.016916         1
1  6531  164.460412    2429.508343         1
2  4555  191.616081   46088.726420         1
3  8497  210.496795    2990.580635         1
4  2096  842.906821  194922.480200         1


In [33]:
def newStrata(df):    
    N = df.coID.count()
    N_h = df.groupby(['strata_2'], as_index=False).count()['coID']
    s_h = df.groupby(['strata_2'], as_index=False).std()['sales']
    
    k = 1 / (1/s_h).sum()
    
    N_h_new = ((k / s_h) * N)
    N_h_rounded = N_h_new.round().astype('int')
    
    d = {'N_h':N_h,'N_h_new':N_h_new,'N_h_rounded':N_h_rounded}
    equal_Wh_Sh = pd.DataFrame(data=d)
    
    print(equal_Wh_Sh)
    print " N_h Sum: ",N_h.sum(),"\n N_h_new Sum: ",N_h_rounded.sum()
    print "Need to reduce new N_h sum by: ",N_h_rounded.sum()-N_h.sum()
    
    return N_h_rounded

In [34]:
def updateDF(df, N_h):
    strata = []
    for i in range(len(N_h)):
        for j in range(N_h[i]):
            strata.append(i+1) 
    
    df['strata_2'] = pd.Series(strata, index=df.index)
    return df

### Iteration #1

In [35]:
N_h_new = newStrata(df3)

    N_h      N_h_new  N_h_rounded
0  2560  1144.645417         1145
1  1725  4012.620420         4013
2  1988   864.013649          864
3   707  2392.703434         2393
4  1099   762.596059          763
5   740   333.257224          333
6   410   181.933066          182
7   419    68.261899           68
8   114     1.968829            2
 N_h Sum:  9762 
 N_h_new Sum:  9763
Need to reduce new N_h sum by:  1


In [36]:
N_h_new[4] -= 1
N_h_new.sum()

9762

In [37]:
df3 = updateDF(df3, N_h_new)

### Iteration #2

In [38]:
N_h_new = newStrata(df3)

    N_h      N_h_new  N_h_rounded
0  1145  2607.462897         2607
1  4013  2028.186264         2028
2   864  3768.806612         3769
3  2393   802.260893          802
4   762   284.981401          285
5   333   196.464496          196
6   182    64.742294           65
7    68     8.253215            8
8     2     0.841928            1
 N_h Sum:  9762 
 N_h_new Sum:  9761
Need to reduce new N_h sum by:  -1


In [39]:
N_h_new[5] += 1
N_h_new.sum()

9762

In [40]:
df3 = updateDF(df3, N_h_new)

### Iteration #3

In [44]:
df3['strata_2'].iloc[-1] -= 1
N_h_new = newStrata(df3)

    N_h      N_h_new  N_h_rounded
0  2607  2955.325725         2955
1  2028  5150.802412         5151
2  3769   882.749866          883
3   802   360.813654          361
4   285   305.802532          306
5   197    97.137197           97
6    66     4.373593            4
7     8     4.995020            5
 N_h Sum:  9762 
 N_h_new Sum:  9762
Need to reduce new N_h sum by:  0


In [45]:
df3 = updateDF(df3, N_h_new)

### Iteration #4

In [46]:
N_h_new = newStrata(df3)

    N_h      N_h_new  N_h_rounded
0  2955  5733.379024         5733
1  5151  1614.930764         1615
2   883  1125.213137         1125
3   361   811.199110          811
4   306   361.365384          361
5    97    78.085515           78
6     4    34.816209           35
7     5     3.010857            3
 N_h Sum:  9762 
 N_h_new Sum:  9761
Need to reduce new N_h sum by:  -1


In [47]:
N_h_new[4] += 1
N_h_new.sum()

9762

In [48]:
df3 = updateDF(df3, N_h_new)

### Iteration #5

In [49]:
N_h_new = newStrata(df3)

    N_h      N_h_new  N_h_rounded
0  5733  2466.219535         2466
1  1615  3935.406885         3935
2  1125  2428.758336         2429
3   811   474.130732          474
4   362   300.268337          300
5    78   135.214952          135
6    35    19.926503           20
7     3     2.074719            2
 N_h Sum:  9762 
 N_h_new Sum:  9761
Need to reduce new N_h sum by:  -1


In [50]:
N_h_new[1] += 1
N_h_new.sum()

9762

In [51]:
df3 = updateDF(df3, N_h_new)

### Iteration #6

In [52]:
N_h_new = newStrata(df3)

    N_h      N_h_new  N_h_rounded
0  2466  4987.571036         4988
1  3936  2480.159284         2480
2  2429  1122.054469         1122
3   474   636.559003          637
4   300   439.843068          440
5   135    81.044572           81
6    20    12.832694           13
7     2     1.935873            2
 N_h Sum:  9762 
 N_h_new Sum:  9763
Need to reduce new N_h sum by:  1


In [53]:
N_h_new[3] -= 1
N_h_new.sum()

9762

In [54]:
df3 = updateDF(df3, N_h_new)

### Iteration #7

In [55]:
N_h_new = newStrata(df3)

    N_h      N_h_new  N_h_rounded
0  4988  3637.886716         3638
1  2480  2825.738691         2826
2  1122  2321.719416         2322
3   636   643.025777          643
4   440   231.412194          231
5    81    88.914115           89
6    13    11.373774           11
7     2     1.929316            2
 N_h Sum:  9762 
 N_h_new Sum:  9762
Need to reduce new N_h sum by:  0


In [56]:
df3 = updateDF(df3, N_h_new)

### Iteration #8

In [57]:
N_h_new = newStrata(df3)

    N_h      N_h_new  N_h_rounded
0  3638  4947.010396         4947
1  2826  2731.238277         2731
2  2322  1238.075513         1238
3   643   426.893539          427
4   231   324.764426          325
5    89    80.904074           81
6    11    11.149031           11
7     2     1.964745            2
 N_h Sum:  9762 
 N_h_new Sum:  9762
Need to reduce new N_h sum by:  0


In [58]:
df3 = updateDF(df3, N_h_new)

### Iteration #9

In [59]:
N_h_new = newStrata(df3)

    N_h      N_h_new  N_h_rounded
0  4947  4420.191468         4420
1  2731  2922.206844         2922
2  1238  1274.855827         1275
3   427   742.153704          742
4   325   289.428086          289
5    81    97.783608           98
6    11    13.076116           13
7     2     2.304347            2
 N_h Sum:  9762 
 N_h_new Sum:  9761
Need to reduce new N_h sum by:  -1


In [60]:
N_h_new[4] += 1
N_h_new.sum()

9762

In [61]:
df3 = updateDF(df3, N_h_new)

### Iteration #10

In [62]:
N_h_new = newStrata(df3)

    N_h      N_h_new  N_h_rounded
0  4420  4411.607753         4412
1  2922  2440.218222         2440
2  1275  2007.048796         2007
3   742   455.419781          455
4   290   354.868991          355
5    98    79.825009           80
6    13    11.124428           11
7     2     1.887020            2
 N_h Sum:  9762 
 N_h_new Sum:  9762
Need to reduce new N_h sum by:  0


In [63]:
df3 = updateDF(df3, N_h_new)

### Iteration #11

In [64]:
N_h_new = newStrata(df3)

    N_h      N_h_new  N_h_rounded
0  4412  4579.381114         4579
1  2440  3095.940275         3096
2  2007  1110.920799         1111
3   455   644.112400          644
4   355   235.438550          235
5    80    83.161842           83
6    11    11.090576           11
7     2     1.954444            2
 N_h Sum:  9762 
 N_h_new Sum:  9761
Need to reduce new N_h sum by:  -1


In [65]:
N_h_new[4] += 1
N_h_new.sum()

9762

In [66]:
df3 = updateDF(df3, N_h_new)

### Iteration #12

In [67]:
N_h_new = newStrata(df3)

    N_h      N_h_new  N_h_rounded
0  4579  4781.231882         4781
1  3096  2468.391973         2468
2  1111  1637.050660         1637
3   644   465.790613          466
4   236   304.379269          304
5    83    90.798462           91
6    11    12.206110           12
7     2     2.151030            2
 N_h Sum:  9762 
 N_h_new Sum:  9761
Need to reduce new N_h sum by:  -1


In [68]:
N_h_new[4] += 1
N_h_new.sum()

9762

In [69]:
df3 = updateDF(df3, N_h_new)

### Iteration #13

In [70]:
N_h_new = newStrata(df3)

    N_h      N_h_new  N_h_rounded
0  4781  4305.518437         4306
1  2468  3219.488896         3219
2  1637  1173.599929         1174
3   466   630.354881          630
4   305   330.547546          331
5    91    88.295189           88
6    12    12.101467           12
7     2     2.093655            2
 N_h Sum:  9762 
 N_h_new Sum:  9762
Need to reduce new N_h sum by:  0


In [71]:
df3 = updateDF(df3, N_h_new)

### Iteration #14

In [72]:
N_h_new = newStrata(df3)

    N_h      N_h_new  N_h_rounded
0  4306  4735.049515         4735
1  3219  2255.675728         2256
2  1174  1844.047953         1844
3   630   543.510708          544
4   331   285.986858          286
5    88    84.372461           84
6    12    11.386771           11
7     2     1.970007            2
 N_h Sum:  9762 
 N_h_new Sum:  9762
Need to reduce new N_h sum by:  0


In [73]:
df3 = updateDF(df3, N_h_new)

### Iteration #15

In [74]:
N_h_new = newStrata(df3)

    N_h      N_h_new  N_h_rounded
0  4735  4208.355163         4208
1  2256  3435.084159         3435
2  1844  1212.230627         1212
3   544   528.932090          529
4   286   279.455897          279
5    84    84.534445           85
6    11    11.398850           11
7     2     2.008770            2
 N_h Sum:  9762 
 N_h_new Sum:  9761
Need to reduce new N_h sum by:  -1


In [75]:
N_h_new[4] += 1
N_h_new.sum()

9762

In [76]:
df3 = updateDF(df3, N_h_new)

In [77]:
df['strata_3'] = pd.Series(df3['strata_2'], index=df.index)
print df.head(5)

   coID       sales      inventory  strata_1  strata_2  strata_3
0  6578  124.430200    6756.016916         1         1         1
1  6531  164.460412    2429.508343         1         1         1
2  4555  191.616081   46088.726420         1         1         1
3  8497  210.496795    2990.580635         1         1         1
4  2096  842.906821  194922.480200         1         1         1


In [82]:
df.to_csv('projectData2.csv')

## Neyman Allocation for Methods

In [84]:
def neymanAlloc(df, strata_field, sample_size):
    neyman_alloc = pd.DataFrame()
    neyman_alloc['stratum'] = df.groupby([strata_field], as_index = False).count()[strata_field]
    neyman_alloc['N_h'] = df.groupby([strata_field], as_index = False).count()['coID']
    neyman_alloc['N_h*S_h'] = df.groupby([strata_field], as_index = False).std()['sales'] * neyman_alloc['N_h']
    neyman_alloc['N_h*S_h/sum(N_h*S_h)'] = neyman_alloc['N_h*S_h'] / neyman_alloc['N_h*S_h'].sum()
    neyman_alloc['samp*N_h*S_h/sum(N_h*S_h)'] = sample_size * neyman_alloc['N_h*S_h/sum(N_h*S_h)']
    neyman_alloc['sample size'] = neyman_alloc['samp*N_h*S_h/sum(N_h*S_h)'].round().astype('int')
    
    print neyman_alloc
    return neyman_alloc['N_h'], neyman_alloc['sample size']

### Method #1

In [85]:
strata_1_Nh, strata_1_neyman = neymanAlloc(df, 'strata_1', 500)

   stratum   N_h       N_h*S_h  N_h*S_h/sum(N_h*S_h)  \
0        1  4285  4.146658e+07              0.109948   
1        2  1988  2.395574e+07              0.063518   
2        3  1308  1.322527e+07              0.035067   
3        4   919  1.366197e+07              0.036225   
4        5   554  2.464330e+07              0.065341   
5        6   314  1.848615e+07              0.049016   
6        7   201  1.435766e+07              0.038069   
7        8   125  4.080337e+07              0.108190   
8        9    57  3.226548e+07              0.085552   
9       10    11  1.542808e+08              0.409074   

   samp*N_h*S_h/sum(N_h*S_h)  sample size  
0                  54.974127           55  
1                  31.759218           32  
2                  17.533338           18  
3                  18.112287           18  
4                  32.670743           33  
5                  24.507930           25  
6                  19.034595           19  
7                  54.094881   

In [89]:
for i in range(len(strata_1_Nh)-1,-1,-1):
    if strata_1_Nh[i] < strata_1_neyman[i]:
        strata_1_neyman[i-1] += strata_1_neyman[i] - strata_1_Nh[i]
        strata_1_neyman[i] = strata_1_Nh[i]
print strata_1_neyman

0     55
1     32
2     18
3     18
4     33
5     25
6    128
7    125
8     57
9     11
Name: sample size, dtype: int64


### Method #2

In [90]:
strata_2_Nh, strata_2_neyman = neymanAlloc(df, 'strata_2', 500)

   stratum   N_h       N_h*S_h  N_h*S_h/sum(N_h*S_h)  \
0        1  2560  2.328536e+07              0.029733   
1        3  1725  4.475844e+06              0.005715   
2        4  1988  2.395574e+07              0.030589   
3        5   707  3.076412e+06              0.003928   
4        6  1099  1.500434e+07              0.019159   
5        7   740  2.311884e+07              0.029521   
6        8   410  2.346314e+07              0.029960   
7        9   419  6.390716e+07              0.081604   
8       10   114  6.028519e+08              0.769789   

   samp*N_h*S_h/sum(N_h*S_h)  sample size  
0                  14.866686           15  
1                   2.857631            3  
2                  15.294700           15  
3                   1.964155            2  
4                   9.579621           10  
5                  14.760373           15  
6                  14.980193           15  
7                  40.801942           41  
8                 384.894700          385  

In [92]:
for i in range(len(strata_2_Nh)-1,-1,-1):
    if strata_2_Nh[i] < strata_2_neyman[i]:
        strata_2_neyman[i-1] += strata_2_neyman[i] - strata_2_Nh[i]
        strata_2_neyman[i] = strata_2_Nh[i]
print strata_2_neyman

0     15
1      3
2     15
3      2
4     10
5     15
6     15
7    312
8    114
Name: sample size, dtype: int64


### Method #3

In [93]:
strata_3_Nh, strata_3_neyman = neymanAlloc(df, 'strata_3', 500)

   stratum   N_h       N_h*S_h  N_h*S_h/sum(N_h*S_h)  \
0        1  4208  4.022307e+07              0.102352   
1        2  3435  7.625276e+07              0.194033   
2        3  1212  4.468617e+07              0.113708   
3        4   529  4.682416e+07              0.119149   
4        5   280  4.566994e+07              0.116212   
5        6    85  4.734790e+07              0.120481   
6        7    11  4.527419e+07              0.115205   
7        8     2  4.671096e+07              0.118861   

   samp*N_h*S_h/sum(N_h*S_h)  sample size  
0                  51.175800           51  
1                  97.016370           97  
2                  56.854203           57  
3                  59.574364           60  
4                  58.105852           58  
5                  60.240724           60  
6                  57.602339           58  
7                  59.430348           59  


In [94]:
for i in range(len(strata_3_Nh)-1,-1,-1):
    if strata_3_Nh[i] < strata_3_neyman[i]:
        strata_3_neyman[i-1] += strata_3_neyman[i] - strata_3_Nh[i]
        strata_3_neyman[i] = strata_3_Nh[i]
print strata_3_neyman

0     51
1     97
2     57
3     60
4    137
5     85
6     11
7      2
Name: sample size, dtype: int64
